In [ ]:
#### Load Libraries
import numpy as np
from joblib import load
from keras.models import Model, load_model
import glob 
import  pandas as pd
import matplotlib.pyplot as plt
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
import os
from pymongo import MongoClient
import plotly.express as ex
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
from datetime import timedelta
from scipy.fft import fft, ifft, rfft, fftfreq

In [ ]:
start= datetime.strptime("2022-07-21T17:56:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000") 
end= datetime.strptime("2022-07-22T06:31:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000") 

In [ ]:
#####DB Config
host= "DevEdgeV32"
port=27017

In [ ]:
### set parameters
data_path= 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/New_Experiments/assets/data/_N_072221LOG_PLA3_MPF.csv'
preprocessor_dir='G:/Innovations@HELLER/DN/KI/Temperaturkompensation/New_Experiments/assets/preprocessor/'
preprocessor_name='preprocessor.p'
model_dir='G:/Innovations@HELLER/DN/KI/Temperaturkompensation/New_Experiments/assets/model/'
model_name= 'model.h5'
#selected_Columns= ['t_spindle', 't_motor','supply_temp']
output_variable= 'welle_z'
selected_Columns=  ['t_bett','t_motor', 't_spindle','DRZ', output_variable]#
prediction_variable= 'prediction'
window=10
shift= 1

In [ ]:
client = MongoClient(host= host, port=port)
db = client.h4ai
event_list = db.modelLogs.find({ "date" : { '$gte' : start, '$lt' : end} }).sort('date', 1)
signals= None
i=0
for event in event_list:
    i+=1
    record_list= event['content']
    keys= None
    for record in record_list:
        keys= record['raw_data'].keys()
        for item in keys:
            record['raw_data'][item]= [record['raw_data'][item]]
        #print ('before',record)
        record['raw_data']['date']=[record['date'] + timedelta(hours= +2)]
        record['raw_data'][output_variable]=[record[output_variable]]
        record['raw_data'][prediction_variable]=[record[prediction_variable]]
        record['raw_data']['DRZ'][0]= float(record['raw_data']['DRZ'][0])
        #record['raw_data']['M8']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        #record['raw_data']['M121']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        #record['raw_data']['M127']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        #record['raw_data']['M7']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['t_spindle'][0]= np.round(record['raw_data']['t_spindle'][0] - record['raw_data']['t_bett'][0], decimals=2)
        # record['raw_data']['t_motor']= np.round(float(record['raw_data']['t_motor'][0]) - record['raw_data']['t_bett'][0], decimals= 2)
        record['raw_data']['t_spindle'][0]= np.round(record['raw_data']['t_spindle'][0], decimals=2)
        record['raw_data']['t_motor']= np.round(float(record['raw_data']['t_motor'][0]), decimals= 2)
        #print ('After ',record)
        lf_signal_point= pd.DataFrame(record['raw_data'])
        if signals is None:
            signals= lf_signal_point
        else:
            signals= signals.append(lf_signal_point,ignore_index= True)
signals['t_motor']= signals['t_motor'].apply(np.float32)
signals.reset_index(inplace= True)
output=  signals[output_variable]
machine_prediction= signals[prediction_variable]
rough_signals= signals[selected_Columns].to_numpy()
print(rough_signals.shape)
signals.rename(columns={'t_spindle': 't_spindel', 'DRZ': 'drhz'}, inplace= True)
print (signals.head(3))
db_signals= signals
# print ('count= ', i)
# # signals= signals[713:]
# # signals.to_csv('machine_expriment3.csv')
selected_Columns= ['t_bett' ,'t_motor', 't_spindel','drhz', output_variable]


In [ ]:
db_signals

In [ ]:
## Comparasion between two sources:
y_axis_names= selected_Columns
fig= make_subplots(rows=len(selected_Columns),cols=1,shared_xaxes= True, print_grid= True, subplot_titles= selected_Columns, vertical_spacing=0.02)
for i in range(len(selected_Columns)):
    fig.add_trace(go.Line(x= signals['date'],y= signals[selected_Columns[i]], name=selected_Columns[i]), row= i+1, col= 1)
    fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
fig.update_layout(height=900, width=1400, title_text= 'Prediction Results')
fig.show()

In [ ]:
from datetime import date,timedelta
from numpy import timedelta64
dates= db_signals['date'].to_numpy()
num_samples= len(dates)
duration= (dates[-1] - dates[0])
duration = timedelta64(duration, 's').astype(int)
print(duration)
sampling_rate= 1/1.26  #Hz
n= int(duration * sampling_rate)
print(n)


In [ ]:
def calc_fft(signal_data, n, sampling_rate):
    signal_data_std= signal_data #- np.mean(signal_data)
    data_fft= fft(np.asarray(signal_data_std))
    num_samples= len(signal_data)
    power = np.abs(data_fft[:len(data_fft)])
    freq_axis= fftfreq(n, sampling_rate)
    return freq_axis, power


In [ ]:
for signal_name in selected_Columns:
    signal_data= db_signals[signal_name].to_numpy()
    
    diff_signal= np.asanyarray([0 if i == 0 else signal_data[i] - signal_data[i-1] for i in range(0, len(signal_data))])
    freq_axis, power= calc_fft(signal_data= signal_data, n= num_samples, sampling_rate= sampling_rate)
    freq_axis_diff, power_diff= calc_fft(signal_data= diff_signal, n= num_samples, sampling_rate= sampling_rate)
    fig= make_subplots(rows=4,cols=1,shared_xaxes= False, print_grid= True, vertical_spacing=0.02)
    fig.add_trace(go.Line(x=db_signals['date'],y= signal_data, name=signal_name), row= 1, col= 1)
    fig.update_yaxes(title_text= signal_name, row= 1, col= 1)
    ####Frequency Signal#####
    fig.add_trace(go.Line(x= freq_axis ,y= power, name="FFT"), row= 2, col= 1)
    fig.update_yaxes(title_text= 'FFT', row= 2, col= 1)
    #####Difference
    fig.add_trace(go.Line(x=db_signals['date'],y= diff_signal, name= str('diff' + signal_name)), row= 3, col= 1)
    fig.update_yaxes(title_text= str('diff' + signal_name), row= 3, col= 1)
    ####Frequency Signal#####
    fig.add_trace(go.Line(x= freq_axis_diff,y= power_diff, name="FFT for diff signal"), row= 4, col= 1)
    fig.update_yaxes(title_text= 'FFT', row= 4, col= 1)
    fig.update_layout(height=900, width=1400, title_text= 'Results')
    fig.show()
    ##########
    

In [ ]:
f= 1000
spacing= 1/(f)
num_samples= 500
signal_data= np.asarray([np.sin(f*2*np.pi*x) for x in np.linspace(0,spacing * num_samples,num_samples, endpoint= False)*2*np.pi/360])
signal_data_std= signal_data - np.mean(signal_data)
data_fft= fft(np.asarray(signal_data_std))
#freq_axis= fftfreq(num_samples, spacing)
frequencies = np.abs(data_fft[:len(data_fft)])
freq_axis = [u * (spacing / num_samples) for u in range(len(frequencies))]
# print(len(frequencies))
fig= make_subplots(rows=2,cols=1,shared_xaxes= False, print_grid= True, vertical_spacing=0.02)
fig.add_trace(go.Line(y= signal_data, name=signal_name), row= 1, col= 1)
fig.update_yaxes(title_text= signal_name, row= 1, col= 1)
####Frequency Signal#####
fig.add_trace(go.Line(x= freq_axis ,y= frequencies, name="FFT"), row= 2, col= 1)
fig.update_yaxes(title_text= 'FFT', row= 2, col= 1)
fig.update_layout(height=900, width=1400, title_text= 'Prediction Results')
fig.show()